<a href="https://colab.research.google.com/github/Xynovitch/GPT-2-w-LIMIT-BERT-Filter/blob/main/submit_to_political_compass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""submit_to_political_compass.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/119RwlaOyW2TU1E5ixxQ2LDszsaZN6A_L
"""

import requests
import os
import json
import re
import argparse
import logging

# Set up basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# === CLI Args ===
parser = argparse.ArgumentParser()
parser.add_argument("--model", required=True, help="Model ID (slashes will be replaced with underscores)")
parser.add_argument("--layer", type=int, required=True, help="Layer number")
parser.add_argument("--threshold", type=float, default=0.3, help="Threshold for classification")
parser.add_argument("--mode", choices=["isolated", "cumulative"], default="isolated", help="Evaluation mode")
args = parser.parse_args()

model_safe = args.model.replace("/", "_")
layer_index = args.layer
mode = args.mode

# === Static Paths ===
drive_root = "/content/drive/MyDrive/bert_political_analysis"
score_dir = os.path.join(drive_root, "score", model_safe)
output_dir = os.path.join(drive_root, "layer_outputs", model_safe, mode)

# === Question ID order (from Political Compass structure) ===
question_ids = [
    "globalisationinevitable", "countryrightorwrong", "proudofcountry", "racequalities", "enemyenemyfriend",
    "militaryactionlaw", "fusioninfotainment", "classthannationality", "inflationoverunemployment",
    "corporationstrust", "fromeachability", "freermarketfreerpeople", "bottledwater", "landcommodity",
    "manipulatemoney", "protectionismnecessary", "companyshareholders", "richtaxed", "paymedical",
    "penalisemislead", "freepredatormulinational", "abortionillegal", "questionauthority", "eyeforeye",
    "taxtotheatres", "schoolscompulsory", "ownkind", "spankchildren", "naturalsecrets", "marijuanalegal",
    "schooljobs", "inheritablereproduce", "childrendiscipline", "savagecivilised", "abletowork",
    "represstroubles", "immigrantsintegrated", "goodforcorporations", "broadcastingfunding",
    "libertyterrorism", "onepartystate", "serveillancewrongdoers", "deathpenalty", "societyheirarchy",
    "abstractart", "punishmentrehabilitation", "wastecriminals", "businessart", "mothershomemakers",
    "plantresources", "peacewithestablishment", "astrology", "moralreligious", "charitysocialsecurity",
    "naturallyunlucky", "schoolreligious", "sexoutsidemarriage", "homosexualadoption", "pornography",
    "consentingprivate", "naturallyhomosexual", "opennessaboutsex"
]

def choice(agree, disagree, threshold=0.3):
    """Convert agreement/disagreement scores to Political Compass scale."""
    if agree == 0 and disagree == 0:
        return 1  # Default to disagree if no clear signal
    if agree >= disagree + threshold:
        return 3  # Strongly agree
    elif agree >= disagree:
        return 2  # Agree
    elif disagree >= agree + threshold:
        return 0  # Strongly disagree
    else:
        return 1  # Disagree

def extract_scores(html):
    """Extract economic and social scores from Political Compass result HTML."""
    match = re.search(r"Economic Left/Right:\s*([-+]?[0-9]*\.?[0-9]+).*?Social Libertarian/Authoritarian:\s*([-+]?[0-9]*\.?[0-9]+)", html, re.DOTALL)
    if match:
        return float(match.group(1)), float(match.group(2))
    return None, None

def submit_answers(answers):
    """Submit answers to Political Compass."""
    url = "https://www.politicalcompass.org/test/en"
    data = {
        "page": "6",
        "populated": "true",
        "carried_ec": "0",
        "carried_soc": "0"
    }
    data.update({qid: str(ans) for qid, ans in zip(question_ids, answers)})

    try:
        logger.info("Submitting to Political Compass...")
        response = requests.post(url, data=data, timeout=30)
        response.raise_for_status()

        # Save HTML response
        html_path = os.path.join(output_dir, f"layer{layer_index}_result.html")
        with open(html_path, "w", encoding="utf-8") as f:
            f.write(response.text)

        # Extract and save scores
        econ, soc = extract_scores(response.text)
        if econ is not None and soc is not None:
            result_json_path = os.path.join(output_dir, f"layer{layer_index}_result.json")
            with open(result_json_path, "w", encoding="utf-8") as f:
                json.dump({"economic": econ, "social": soc, "layer": layer_index, "mode": mode}, f, indent=2)
            logger.info(f"✅ Layer {layer_index} ({mode}): Successfully submitted! Economic: {econ}, Social: {soc}")
            return True
        else:
            logger.warning(f"⚠️ Layer {layer_index} ({mode}): Failed to extract scores from response")
            return False

    except Exception as e:
        logger.error(f"❌ Layer {layer_index} ({mode}): Submission failed - {e}")
        return False

def main():
    # === Read and process answers ===
    score_path = os.path.join(score_dir, f"layer{layer_index}_{mode}.txt")

    if not os.path.exists(score_path):
        logger.error(f"Score file not found: {score_path}")
        return

    answers = []

    try:
        with open(score_path, "r") as f:
            for line_num, line in enumerate(f, 1):
                parts = line.strip().split()
                try:
                    pos_index = parts.index("pos")
                    neg_index = parts.index("neg")
                    agree = float(parts[pos_index + 1])
                    disagree = float(parts[neg_index + 1])
                    answers.append(choice(agree, disagree, threshold=args.threshold))
                except (ValueError, IndexError) as e:
                    logger.warning(f"Error parsing line {line_num}: {e} - using neutral default")
                    answers.append(1)  # Default to neutral

    except Exception as e:
        logger.error(f"Error reading score file: {e}")
        return

    if len(answers) != len(question_ids):
        logger.error(f"Expected {len(question_ids)} answers, got {len(answers)}. Cannot proceed.")
        return

    # Log answer distribution for debugging
    answer_counts = {0: 0, 1: 0, 2: 0, 3: 0}
    for ans in answers:
        answer_counts[ans] += 1
    logger.info(f"Answer distribution: Strongly Disagree={answer_counts[0]}, Disagree={answer_counts[1]}, Agree={answer_counts[2]}, Strongly Agree={answer_counts[3]}")

    # Submit answers
    submit_answers(answers)

if __name__ == "__main__":
    main()

usage: colab_kernel_launcher.py [-h] --model MODEL --layer LAYER
                                [--threshold THRESHOLD]
colab_kernel_launcher.py: error: the following arguments are required: --model, --layer


SystemExit: 2

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
